


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

- - - 

# Imports and data


### Dados

Os dados são [geladeiras](https://www.magazineluiza.com.br/geladeira-refrigerador/eletrodomesticos/s/ed/refr/) e [microondas](https://www.magazineluiza.com.br/lava-e-seca/eletrodomesticos/s/ed/ela1/) crawleados da magazine luiza :)

Código [aqui](https://github.com/BrunoGomesCoelho/pandas-soup/blob/master/src/scraping.py).


In [13]:
import pandas as pd
import pickle

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

WASHER_FILE = "washers.csv"
REFRIGERATORS_FILE = "refrigerators.csv"
TEST_SIZE = 0.3

def load_files():
    washers = pd.read_csv(WASHER_FILE)
    refrigerators = pd.read_csv(REFRIGERATORS_FILE)

    washers["Category"] = 0
    refrigerators["Category"] = 1

    return washers, refrigerators

In [2]:
washers, refrigerators = load_files()

In [3]:
washers.head()

,Code,Title,Brand,Model,Price,Category
0,105418,"""Lava e Seca LG 11kg Smart Care Prime Touch ""","""lg""","""14 Programas de Lavagem Água Quente""",2799.9,0
1,2190089,"""Lava e Seca Samsung 10,2kg Prata WD4000""","""samsung""","""12 Programas de Lavagem Água Quente""",2699.9,0
2,2190087,"""Lava e Seca Samsung 10,2kg WD4000""","""samsung""","""12 Programas de Lavagem Água Quente""",2449.9,0
3,2159005,"""Lava e Seca Samsung 10,2kg Inox WD6000""","""samsung""","""13 Programas de Lavagem""",3299.9,0
4,2190566,"""Lava e Seca Midea 12Kg Prata Acqua""","""midea""","""16 Programas de Lavagem Água Quente""",2599.9,0





<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

- - - 

# Encodar os dados

In [14]:
def encode_data(washers, refrigerators, drop_cols):
    categorical_labels = ["Title", "Brand", "Model", "Category"]
    all_data = pd.concat([washers, refrigerators])

    # Transform categorical data to numeric for xgboost
    for label in categorical_labels:
        if label in drop_cols:
            continue
        washers.loc[washers[label].isnull(), label] = "nullo"
        refrigerators.loc[refrigerators[label].isnull(), label] = "nullo"
        all_data.loc[all_data[label].isnull(), label] = "nullo"
    
        encoding = LabelEncoder()
        encoding.fit(all_data[label])
        washers[label] = encoding.transform(washers[label])
        refrigerators[label] = encoding.transform(refrigerators[label])
        
        pickle.dump(encoding, open(label, "wb"))
        
    return washers, refrigerators, all_data




<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

- - - 

# Rodar o K-NN com K = 1


In [24]:
def main():
    washers, refrigerators = load_files()
    drop_cols = ["Title", "Model", "Code"]
    washers.drop(columns=drop_cols, inplace=True)
    refrigerators.drop(columns=drop_cols, inplace=True)
    

    washers, refrigerators, all_data = encode_data(washers, refrigerators,
                                                  drop_cols)

    # Deal with None price
    label = "Price"
    washers.loc[washers[label] == "None"] = 0
    refrigerators.loc[refrigerators[label] == "None"] = 0

    # Convert string price to float
    washers["Price"] = washers["Price"].astype(float)
    refrigerators["Price"] = refrigerators["Price"].astype(float)

    # Free some RAM due to my laptop lagging
    del all_data

    # We split data into train/test
    washer_train, washer_test = train_test_split(washers, test_size=TEST_SIZE,
                                                random_state=2)
    refrigerators_train, refrigerators_test = train_test_split(refrigerators, 
                                                               test_size=TEST_SIZE,
                                                              random_state=2)

    # Merge the differents products
    train = pd.concat([washer_train, refrigerators_train])
    test = pd.concat([washer_test, refrigerators_test])

    # Laptop limitations
    del washers, refrigerators, washer_train, washer_test,\
        refrigerators_train, refrigerators_test

    # Get prediction
    y_train = train["Category"].copy()
    train.drop("Category", axis=1, inplace=True)

    y_test = test["Category"].copy()
    test.drop("Category", axis=1, inplace=True)

    clf = KNeighborsClassifier(n_neighbors=1)

    # A better idea would be having a validation to use as a eval set.
    #   Since the problem is quite easy, we skipped this
    clf.fit(train, y_train)

    # make predictions for test data
    y_pred = clf.predict(test)

    # evaluate predictions
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    
    return clf

if __name__ == "__main__":
    clf = main()

Accuracy: 78.82%





<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

- - - 

# Salvar


In [25]:
import pickle
pickle.dump(clf, open("classificador", "wb"))